<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="64"/>

# <span style="color: #336699">Land use and land cover classification in the Brazilian Cerrado biome using Brazil Data Cube</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Rolf E. O. Simões <sup><a href="mailto:rolf.simoes@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-0953-4132"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Alber H. Sanchez <sup><a href="mailto:alber.ipia@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0001-7966-2880"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe M. Carlos <sup><a href="mailto:felipe.carlos@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Leonardo S. Vieira <sup><a href="mailto:leonardo.vieira@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0002-3397-6232"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>,<br/>
    Karine R. Ferreira <sup><a href="mailto:karine.ferreira@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Lubia Vinhas <sup><a href="mailto:lubia.vinhas@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-1104-3607"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup>* <a href="mailto:gilberto.queiroz@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    <sup>*</sup> Author to whom correspondence should be addressed.
    <br/><br/>
    February 24, 2021
</div>

<br/>

<div style="text-align: justify;  margin-left: 10%; margin-right: 10%;">
<b>Abstract.</b> This Jupyter Notebook compendium contains useful information for the creation of land use and land cover (LULC) maps using Earth observations data cubes and machine learning (ML) techniques. The code is based on the research pipeline described in the paper <em>Earth Observation Data Cubes for Brazil: Requirements, Methodology and Products</em>. These notebooks access open data available in the Brazil Data Cube platform.
</div>    

<br/>
<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is supplement to the <a href="https://www.mdpi.com/2072-4292/12/24/4033/htm#sec5-remotesensing-12-04033" target="_blank">Section 5</a> of the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Ferreira, K.R.; Queiroz, G.R.; Vinhas, L.; Marujo, R.F.B.; Simoes, R.E.O.; Picoli, M.C.A.; Camara, G.; Cartaxo, R.; Gomes, V.C.F.; Santos, L.A.; Sanchez, A.H.; Arcanjo, J.S.; Fronza, J.G.; Noronha, C.A.; Costa, R.W.; Zaglia, M.C.; Zioti, F.; Korting, T.S.; Soares, A.R.; Chaves, M.E.D.; Fonseca, L.M.G. 2020. Earth Observation Data Cubes for Brazil: Requirements, Methodology and Products. Remote Sens. 12, no. 24: 4033. DOI: <a href="https://doi.org/10.3390/rs12244033" target="_blank">10.3390/rs12244033</a>.
    </div>
</div>

# <span style="color: #336699">Land Use and Cover Mapping from Sentinel-2/MSI Data Cubes</span>
<hr style="border:1px solid #0077b9;">

This document will present the steps to create a LULC map based on Sentinel-2/MSI data cube, training samples and a MultiLayer Perceptron neural network. This code relies on the [SITS R package](https://github.com/e-sensing/sits).

## <span style="color: #336699">Study Area and samples</span>
<hr style="border:0.5px solid #0077b9;">

In this application we use the same region of interest and samples described in [Ferreira et al. (2020)](https://doi.org/10.3390/rs12244033). As depicted in Figure 1, the region is located in the Bahia state (Brazil), between the Cerrado and Caatinga biomes.


<div align="center">
  <img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/bdc-article/study-area.png" width="600px">
</div>
<br/>
<center><b>Figure 1</b> - Study area in relation to Brazil and its biomes.</center>

## <span style="color: #336699">Set a pseudo-randomic seed</span>
<hr style="border:0.5px solid #0077b9;">

We will fix a pseudo-randomic seed in order to run the code:

In [ ]:
set.seed(777)

## <span style="color: #336699">Loading the software packages</span>
<hr style="border:0.5px solid #0077b9;">

In [ ]:
library(sits)

The user should also provides his access key to the Brazil Data Cube platform:

In [ ]:
MY_ACCESS_KEY <- "change-me"
Sys.setenv(BDC_ACCESS_KEY = MY_ACCESS_KEY)

## <span style="color: #336699">Defining the Data Cube</span>
<hr style="border:0.5px solid #0077b9;">

Let's start by defining the region of interest (ROI) as a sub-space of the red rectangle shown in Figure 1. 

The ROI is available in a file name `roi.rds` under the directory `roi`:

In [ ]:
roi <- readRDS(url("https://brazildatacube.dpi.inpe.br/geo-knowledge-hub/bdc-article/roi/roi.rds"))

> The `roi` is a list with two components:
> * `classification_roi`: contains the geometry boundary for the classification.
> * `search_roi`: a smaller rectangle than the `classification_roi`, that intersects only the data cube tiles we are interested to use in the classification.

Next we define a time interval based on the crop calendar to define the working period:

In [ ]:
start_date  <- "2018-09-01"
end_date    <- "2019-08-31"

In this Jupyter Notebook we focus the classification based on a Sentinel-2/MSI data cube named `S2_10_16D_STK-1`:

In [ ]:
collection  <- "S2_10_16D_STK-1"

Finally, let's define the data cube.

The `sits` package will access the Sentinel-2/MSI data cube available in the Brazil Data Cube platform through the STAC web service:

In [ ]:
cube <- sits_cube(
  type        = "BDC",
  name        = "cube_to_classify",
  url         = "https://brazildatacube.dpi.inpe.br/stac/",
  collection  = collection,
  start_date  = start_date,
  end_date    = end_date,
  roi         = roi$search_roi
)

> The definition above includes the spectral bands `Red`, `Green`, `Blue`, `Near-Infrared (NIR)` and the vegetation indices `EVI` and `NDVI` already available in the cube.

> It also limits the temporal extension to `2018-09` to `2019-08`.

## <span style="color: #336699">Loading the Training Samples</span>
<hr style="border:0.5px solid #0077b9;">

Now, let's load the samples from a prepared file named `S2_10_16D_STK_1.rds`:

In [ ]:
samples <- readRDS(url("https://brazildatacube.dpi.inpe.br/geo-knowledge-hub/bdc-article/training-samples/rds/S2_10_16D_STK_1.rds"))

> The Jupyter Notebook entitled [Extracting time series from sample locations](./01_ExtractTimeSeries.ipynb) describes in detail how to prepare this file.

## <span style="color: #336699">MultiLayer Perceptron model definition</span>
<hr style="border:0.5px solid #0077b9;">

For the classification of data cubes, the article presents the use of an MLP network with five hidden layers with 512 neurons, trained with the backpropagation algorithm, using the Adam optimizer. The model uses the ReLu activation function.

Below is the definition of this model using the [SITS package](https://github.com/e-sensing/sits).

In [ ]:
mlp_model <- sits_deeplearning(layers        = c(512, 512, 512, 512, 512),
                               activation    = "relu",
                               optimizer     = keras::optimizer_adam(lr = 0.001),
                               epochs        = 200)

Below, the defined model is trained using the same samples used in the article.

dl_model <- sits_train(samples, mlp_model)

## <span style="color: #336699">Output Directory</span>
<hr style="border:0.5px solid #0077b9;">

All the results generated in this document will be saved in your user's home directory, inside `results/S2_10_16D_STK_1` directory:

In [ ]:
output_dir <- "results/S2_10_16D_STK_1"

dir.create(
    path         = output_dir, 
    showWarnings = FALSE, 
    recursive    = TRUE
)

## <span style="color: #336699">Classifying the tiles from the data cube</span>
<hr style="border:0.5px solid #0077b9;">

Before running the classification step, you should define the hardware resources that `sits` will be allowed to use during the classification:

In [ ]:
classification_memsize    <- 8 # GB
classification_multicores <- 3 # CPU logical cores

The next cell uses the trainned MLP model (`dl_model`) to perform a classification based on the temporal data from the data cube: 

> This is a time-consuming process.

In [ ]:
probs <- sits_classify(data       = cube,
                       ml_model   = dl_model,
                       memsize    = classification_memsize,
                       multicores = classification_multicores,
                       roi        = roi$classification_roi,
                       output_dir = output_dir)

> The generated data will be stored under the directory indicated by `output_dir`.

> Note that here we use a geometry boundary from `roi$classification_roi` that is smaller than the region defined by the samples.

The classification output of the Multilayer Perceptron (MLP) model is a raster with three layers (one for each land use class) containing the probabilities of each pixel belonging to each of the classes.

For the Sentinel-2/MSI data, three rasters will be generated (`cube_to_classify_088097_probs_2018_8_2019_7_v1.tif`, `cube_to_classify_089097_probs_2018_8_2019_7_v1.tif` and `cube_to_classify_089098_probs_class_2018_8_2019_7_v1.tif`). Each raster has 3 layers containing scaled probabilities (`x 10,000`) corresponding to the classes `Crop` (layer 1), `Natural Vegetation` (layer 2), and `Pasture` (layer 3). 

## <span style="color: #336699">Generating the Thematic Map</span>
<hr style="border:0.5px solid #0077b9;">

We are going to apply a probability Bayesian smoother method over the output of the MLP. This procedure uses the information of a pixel’s neighborhood to update its probabilities by taking the maximum likelihood estimator. The smoothing procedure removes isolated pixel class values and produces more homogeneous spatial areas.

The next cell perform this operation:

In [ ]:
probs_smoothed <- sits_smooth(probs, type = "bayes", output_dir = output_dir)

> The above cell will create three rasters: `cube_to_classify_088097_probs_2018_8_2019_7_bayes_v1.tif`, `cube_to_classify_089097_probs_2018_8_2019_7_bayes_v1.tif` and `cube_to_classify_089098_probs_2018_8_2019_7_bayes_v1.tif`.

After that, to generate the thematic maps the most probable class is taken as the pixel class. Each class is represented by the codes 1 (Crop), 2 (Natural Vegetation), and 3 (Pasture). The next cell show how to perform this step:

In [ ]:
labels <- sits_label_classification(probs_smoothed, output_dir = output_dir)

> The final map is represented in three files: `cube_to_classify_088097_probs_class_2018_8_2019_7_v1.tif`, `cube_to_classify_089097_probs_class_2018_8_2019_7_v1.tif` and `cube_to_classify_089098_probs_class_2018_8_2019_7_v1.tif`

## <span style="color: #336699">Visualizing the Thematic Map</span>
<hr style="border:0.5px solid #0077b9;">

The result of the classification is divided into three raster files. This is because for Sentinel-2/MSI the classified region covers three tiles of the BDC.

For the visualization of the complete map, below the three results are joined into a single raster.

In [ ]:
gdalUtils::mosaic_rasters(c(
    paste0(output_dir, "/cube_to_classify_088097_probs_class_2018_8_2019_7_v1.tif"),
    paste0(output_dir, "/cube_to_classify_089097_probs_class_2018_8_2019_7_v1.tif"),
    paste0(output_dir, "/cube_to_classify_089098_probs_class_2018_8_2019_7_v1.tif")
), paste0(output_dir, "/cube_to_classify_merged_probs_class_2018_8_2019_7_v1.tif"))

plot(
    raster::raster(paste0(output_dir, "/cube_to_classify_merged_probs_class_2018_8_2019_7_v1.tif"))
)